# 🎬 AI Video Dubbing - Google Colab Deployment

This notebook deploys your Flask app on Google Colab with **FREE GPU**!

## ⚠️ IMPORTANT: Enable GPU First!
1. Go to **Runtime** → **Change runtime type**
2. Select **T4 GPU** under Hardware accelerator
3. Click **Save**

---


In [ ]:
# Step 1: Check GPU is Available
!nvidia-smi

import torch
print(f"\n✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("❌ No GPU detected! Please enable GPU in Runtime settings.")


## Step 2: Mount Google Drive & Upload Project

Upload your project ZIP file to Google Drive first, then mount it here.


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted!")


In [ ]:
# Upload ZIP directly (Alternative to Google Drive)
# This will open a file picker - select your project ZIP file
from google.colab import files
import zipfile
import os

print("📁 Please upload your project ZIP file...")
uploaded = files.upload()

# Extract the uploaded zip
for filename in uploaded.keys():
    print(f"\n📦 Extracting {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('/content/')
    os.remove(filename)
    print("✅ Extraction complete!")

!ls -la /content/


In [ ]:
# Navigate to project directory
# IMPORTANT: Change folder name if different!
import os

PROJECT_DIR = "/content/lipsyncExecution"  # Change if your folder has different name

# If using Google Drive instead:
# PROJECT_DIR = "/content/drive/MyDrive/lipsyncExecution"

os.chdir(PROJECT_DIR)
print(f"📂 Working directory: {os.getcwd()}")
!ls -la


## Step 3: Install Dependencies

This will take 5-10 minutes. Be patient! ☕


In [ ]:
# Install system dependencies
!apt-get update -qq
!apt-get install -qq ffmpeg libgl1-mesa-glx
print("✅ System dependencies installed!")


In [ ]:
# Install Core Python dependencies
!pip install -q flask flask-login flask-sqlalchemy
!pip install -q moviepy openai-whisper soundfile librosa
!pip install -q transformers accelerate huggingface-hub sentencepiece
!pip install -q opencv-python face-alignment scikit-image ffmpeg-python
!pip install -q pydub numba resampy requests protobuf
!pip install -q pyngrok  # For exposing Flask to internet

print("\n✅ Core dependencies installed!")


In [ ]:
# Install LatentSync dependencies (for lip-sync)
!pip install -q diffusers==0.32.2 omegaconf einops decord
!pip install -q mediapipe python_speech_features scenedetect
!pip install -q imageio imageio-ffmpeg lpips kornia
!pip install -q onnxruntime-gpu insightface

print("\n✅ LatentSync dependencies installed!")


In [ ]:
# Install Parler-TTS (for text-to-speech)
!pip install -q git+https://github.com/huggingface/parler-tts.git

print("\n✅ Parler-TTS installed!")


## Step 4: Setup ngrok (Get Public URL)

ngrok creates a **public URL** so you can access your app from anywhere!

### Get FREE ngrok Token:
1. Go to https://ngrok.com/ and sign up (free)
2. Copy your token from: https://dashboard.ngrok.com/get-started/your-authtoken
3. Paste it in the cell below


In [ ]:
# Configure ngrok - PASTE YOUR TOKEN BELOW!
from pyngrok import ngrok

# ⬇️ PASTE YOUR NGROK TOKEN HERE ⬇️
NGROK_TOKEN = ""  # Example: "2abc123def456..."

if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)
    print("✅ ngrok authenticated!")
else:
    print("⚠️ No ngrok token set!")
    print("   Get your FREE token at: https://dashboard.ngrok.com/get-started/your-authtoken")
    print("   Then paste it in the NGROK_TOKEN variable above and re-run this cell.")


In [ ]:
# Create required directories
import os

os.makedirs('instance/uploads', exist_ok=True)
os.makedirs('instance/outputs', exist_ok=True)
os.makedirs('instance/wav2lip_assets', exist_ok=True)

print("✅ Directories created!")
!ls -la instance/


## 🚀 Step 5: RUN THE APP!

Run the cell below to start your app. You'll get a **public URL** to access it!


In [ ]:
# 🚀 START THE FLASK APP WITH PUBLIC URL
import os
import sys
from pyngrok import ngrok

# Add project to path
sys.path.insert(0, os.getcwd())

# Import Flask app
from webapp import create_app
app = create_app()

# Set production settings
app.config['SECRET_KEY'] = 'colab-deployment-secret-key'

# Start ngrok tunnel
port = 5000
public_url = ngrok.connect(port)

print("=" * 70)
print("🎬 AI VIDEO DUBBING APP IS NOW RUNNING!")
print("=" * 70)
print(f"\n🌐 PUBLIC URL: {public_url}")
print(f"\n📱 Open the link above in any browser to use the app!")
print("\n⚠️  Keep this cell running! App stops when you stop this cell.")
print("=" * 70)

# Run Flask (this blocks - keep running!)
app.run(host='0.0.0.0', port=port, debug=False, use_reloader=False)


---

## 📥 Download Your Output Videos (Before Session Ends!)

Colab sessions expire. Run this to download your dubbed videos:


In [ ]:
# Download all output videos before session ends!
from google.colab import files
import shutil
import os

output_dir = 'instance/outputs'

if os.path.exists(output_dir) and os.listdir(output_dir):
    # Create ZIP of all outputs
    shutil.make_archive('/content/my_dubbed_videos', 'zip', output_dir)
    
    # Download the ZIP
    files.download('/content/my_dubbed_videos.zip')
    print("✅ Your dubbed videos have been downloaded!")
else:
    print("📁 No output videos found yet. Process some videos first!")


---

## ⚠️ Important Notes

### Session Limits (Free Colab):
- **Idle timeout**: ~90 minutes of inactivity
- **Max runtime**: ~12 hours
- **GPU limits**: May disconnect if overused

### Tips:
1. **Keep browser tab open** - prevents idle timeout
2. **Download outputs frequently** - data is lost when session ends
3. **Use Google Drive** for persistent storage

### Troubleshooting:

| Problem | Solution |
|---------|----------|
| No GPU detected | Runtime → Change runtime type → T4 GPU |
| ngrok error | Get free token at ngrok.com |
| Module not found | Re-run installation cells |
| Session disconnected | Re-run all cells from start |

---

**🎉 Enjoy your FREE GPU-powered video dubbing!**
